# Sistem Penjadwalan Praktikum Otomatis menggunakan Algoritma Genetika dan Tabu Search.

## Import Library

In [1]:
import os
import sys
import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "LabTimetablingAPI.settings")
django.setup()

## Import Model

In [42]:
from scheduling_data.models import Laboratory, Module, Participant, Group, Assistant
from collections import namedtuple

## Data extraction from model

In [199]:
class LaboratoryData:

    def __init__(self, lab = Laboratory):
        self._lab = lab
        self._laboratories = lab.objects.all()

    @property
    def laboratories(self): 
        return self._laboratories.values()
    
    def get_assistant(self,id):
        laboratory = self._laboratories.filter(id=id).first()
        assistants = laboratory.assistants.all()
        return assistants
    
    def get_module(self,id):
        laboratory = self._laboratories.filter(id=id).first()
        modules = laboratory.modules.all()
        return modules
    
class ModuleData:

        def __init__(self, module = Module):
            self._module = module
            self._modules = module.objects.all()
    
        @property
        def modules(self): 
            return self._modules

class ParticipantData:

    def __init__(self, participant = Participant):
        self._participant = participant
        self._participants = participant.objects.all()
    
    @property
    def participants(self):
        return self._participants.values()
    
    def group(self, id):
        group_memberships = self._participants.filter(id=id).first().group_memberships.all()
        groups = []
        for group_membership in group_memberships:
            group = group_membership.group
            groups.append(group)
        return groups
    
class GroupData:
    def __init__(self, group = Group):
        self._group = group
        self._groups = group.objects.all()

    @property
    def groups(self):
        return self._groups.values()
    
    def get_group(self, id):
        group = self._groups.filter(id=id).first()
        return group
    
    def participants(self, id):
        group = self._groups.filter(id=id).first()
        group_memberships = group.group_memberships.all()
        participants = []
        for group_membership in group_memberships:
            participant = group_membership.participant
            participants.append(participant)
        return participants
    
    def get_participant_schedule(self, id):
        participants = self.participants(id)
        participant_schedule = []
        for participant in participants:
            participant_schedule.append(participant.regular_schedule)
        return participant_schedule
    
    def get_group_schedule(self, id):
        participant_schedule = self.get_participant_schedule(id)
        if len(participant_schedule) == 0:
            return {}
        days = participant_schedule[0].keys()
        merged_schedule = {day: {} for day in days}
        for day in days:
            for time_slot in participant_schedule[0][day]:
                is_available = all(schedule[day][time_slot] for schedule in participant_schedule)
                merged_schedule[day][time_slot] = is_available
        return merged_schedule
                
            
class AssistantData:

    def __init__(self, assistant = Assistant):
        self._assistant = assistant
        self._assistants = assistant.objects.all()

    @property
    def assistants(self):
        return self._assistants.values()

    
    def get_assistant(self, id):
        assistant = self._assistants.filter(id=id).first()
        return assistant
    
    def get_assistant_modules(self,id):
        assistant = self._assistant.filter(id=id).first()
        assistant_memberships = assistant.assistant_memberships.all()
        modules = []
        for assistant_membership in assistant_memberships:
            module = assistant_membership.module
            modules.append(module)
        return modules

## Genetic Algorithm

In [4]:
import pandas as pd

In [200]:
# Assigning data to variables
lab_data = LaboratoryData()
module_data = ModuleData()
participant_data = ParticipantData()
group_data = GroupData()
assistant_data = AssistantData()

In [190]:
pd.DataFrame(participant_data.participants)

,id,name,nim,semester_id,regular_schedule
0,5,Setio Ningrum,3332210016,1,"{'Friday': {'Shift1': False, 'Shift2': True, '..."
1,6,Ruth Anandina,3332210051,1,"{'Friday': {'Shift1': False, 'Shift2': True, '..."
2,7,ADHITAMA WIRA YUDHA,3332190029,1,"{'Friday': {'Shift1': True, 'Shift2': True, 'S..."
3,8,FARHAN NUGROHO,3332190072,1,"{'Friday': {'Shift1': True, 'Shift2': True, 'S..."
4,9,BAGAS NURYANTO,3332190080,1,"{'Friday': {'Shift1': False, 'Shift2': True, '..."
...,...,...,...,...,...
102,55,Maulana Ali Akbar,3332210082,1,"{'Friday': {'Shift1': False, 'Shift2': True, '..."
103,65,Naga Tunggal,3332210004,1,"{'Friday': {'Shift1': True, 'Shift2': False, '..."
104,86,Randy eleanor,3332210057,1,"{'Friday': {'Shift1': True, 'Shift2': True, 'S..."
105,106,MAHESA NURUL VIKAR,3332200087,1,"{'Friday': {'Shift1': False, 'Shift2': True, '..."


In [201]:
pd.DataFrame(group_data.get_group_schedule(1))

,Friday,Monday,Tuesday,Saturday,Thursday,Wednesday
Shift1,False,True,True,False,True,False
Shift2,True,True,False,False,False,False
Shift3,False,False,False,True,False,False
Shift4,True,False,False,False,False,True
Shift5,False,True,False,True,True,True
Shift6,True,False,True,True,False,True


Gene Representation for Scheduling Problem

In [202]:
TimeSlot = namedtuple("TimeSlot", ["date", "day", "shift"])

In [205]:
# Gene Representation
# 1. Laboratory
# 2. Module chapter
# 3. Group
# 4. Assistant
# 5. Time Slot : Date, Day, Time
# 6. Availability: json, e.g. {"Friday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": true, "Shift5": true, "Shift6": false}, "Monday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": true, "Shift5": true, "Shift6": false}, "Tuesday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": true, "Shift5": true, "Shift6": false}, "Saturday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": true, "Shift5": false, "Shift6": true}, "Thursday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": true, "Shift5": true, "Shift6": true}, "Wednesday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": false, "Shift5": false, "Shift6": true}}

class Gene:
    def __init__(self, lab, module_chapter, group, assistant, time_slot: TimeSlot):
        self.lab = lab
        self.module_chapter = module_chapter
        self.group = group
        self.assistant = assistant
        self.time_slot = time_slot
        
    def __repr__(self):
        return f"Laboratory: {self.lab}, Module Chapter: {self.module_chapter}, Group: {self.group}, Assistant: {self.assistant}, Time Slot: {self.time_slot}"
    
    def __eq__(self, other):
        return self.lab == other.lab and self.module_chapter == other.module_chapter and self.group == other.group and self.assistant == other.assistant and self.time_slot == other.time_slot
    
    def __hash__(self):
        return hash((self.lab, self.module_chapter, self.group, self.assistant, self.time_slot))
    
    def __str__(self) -> str:
        return f"Laboratory: {self.lab}, Module Chapter: {self.module_chapter}, Group: {self.group}, Assistant: {self.assistant}, Time Slot: {self.time_slot}"
    
    def generate(self):
        return Gene(self.lab, self.module_chapter, self.group, self.assistant, self.time_slot)
    

In [206]:
# Chromosome Representation
# 1. Gene
# 2. Fitness

class Chromosome:
    def __init__(self, genes, fitness):
        self.genes = genes
        self.fitness = fitness
        
    def __repr__(self):
        return f"Genes: {self.genes}, Fitness: {self.fitness}"
    
    def __eq__(self, other):
        return self.genes == other.genes and self.fitness == other.fitness
    
    def __hash__(self):
        return hash((self.genes, self.fitness))
    
    def __str__(self) -> str:
        return f"Genes: {self.genes}, Fitness: {self.fitness}"
    
    def generate(self):
        return Chromosome(self.genes, self.fitness)